# Div Arb Straight Up

**The Goal of this notebook is to locate areas of "straight up arbitrage" where we can gather at least .5% 2-day profit on liquid options**

*We Define "Straight Up Arb" as *Arbitrage from purchasing a put close to expiry where*     
`Div*(1-tax) - Cost_option > Underlying - Strike`   
We define percentage of arbitrage as The Percentage of Arbitrage that we can get from this as   
`Div * (1 - tax) / (Underlying + Cost_Option)`   
Since we can put on a position daily we would be satisfied with anything like a .2% return (keep in mind that our profits only get compounded every 1.5 months since there is a 1.5 month delay before getting payment from the dividend. In fact we may slowly bleed liquidity until dividend payments start coming through).

In [ ]:
#Dividend*(1-tax)/(Strike+Cost)

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from bisect import bisect_left

In [12]:
pd.set_option('display.max_columns', 500)
PATH_TO_DIV = "gs://ucf-data/dividends/"
PATH_TO_OPT = "gs://ucf-data/Options/"
PATH_TO_DAILY = "gs://ucf-data/daily_stocks.csv"

In [132]:
stonks = pd.read_csv(PATH_TO_DAILY)    #pd.read_csv('prupes_stonks.csv')

In [136]:
def get_dividend_df(ticker):
    df = pd.read_csv('{}{}.csv'.format(PATH_TO_DIV,ticker.upper()))
    df['DATE'] = df.apply (lambda row: datetime.strptime(row['exOrEffDate'],"%m/%d/%Y"), axis=1)
    return df
def get_opt_df(ticker):
    opts = pd.read_csv('{}{}.csv'.format(PATH_TO_OPT,ticker.upper()))
    opts['date'] = opts['date'].apply(str)
    opts['DATE'] = opts.apply (lambda row: datetime.strptime(row['date'],"%Y%m%d"), axis=1)
    return opts
def get_daily_df(ticker):
    daily = pd.read_csv('{}{}/{}_day_close_.csv'.format(PATH_TO_DAILY, ticker.upper(), ticker.upper()))
    daily['DATE'] = daily.apply(lambda row: datetime.strptime(row['timestamp'],"%Y-%m-%d"), axis=1)
    return daily
def get_stonks_df(ticker):
    stocks = stonks[stonks['TICKER'] == ticker]
    stocks['date'] = stocks['date'].apply(str)
    stocks['DATE'] = stocks.apply (lambda row: datetime.strptime(row['date'],"%Y%m%d"), axis=1)
    return stocks
def convert_date(s):
    a = datetime.strptime(s,"%d/%m/%Y")
    return a.strftime("%Y%m%d")
def previous_day(dates, current_day):
    prev = current_day - timedelta(days=1)
    #print(prev)
    insertion_point = bisect_left(dates, current_day)-1
    if insertion_point == -1:
        return False
    else:
        return dates[insertion_point]
def gen_dates(df):
    return sorted(datetime.strptime(str(elt), "%Y%m%d") for elt in list(df.date))


In [49]:
get_daily_df("AAPL").head()

,timestamp,open,high,low,close,volume,millis,timestamp.1,time_of_last_tick
0,2000-01-03,3.746,4.018,3.632,3.750,125216004.0,946909800.0,2000-01-03,2000-01-03
1,2000-01-04,3.866,3.951,3.614,3.866,120027600.0,946996200.0,2000-01-04,2000-01-04
2,2000-01-05,3.706,3.949,3.679,3.703,182036428.0,947082600.0,2000-01-05,2000-01-05
3,2000-01-06,3.791,3.822,3.393,3.793,181484828.0,947169000.0,2000-01-06,2000-01-06
4,2000-01-07,3.447,3.608,3.411,3.447,106187200.0,947255400.0,2000-01-07,2000-01-07


In [19]:
get_opt_df("AAPL").head()

,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,optionid,cfadj,am_settlement,contract_size,ss_flag,forward_price,expiry_indicator,root,suffix,cusip,ticker,sic,index_flag,exchange_d,class,issue_type,industry_group,issuer,div_convention,exercise_style,am_set_flag
0,101594,20150630,AAPL 150702C100000,1,20150702,20150630.0,C,100000,25.55,25.65,15,6,1.657665,0.971807,0.004203,0.601158,-90.97760,108201233,1,0,100,0,125.42603,w,NaN,NaN,3783310,AAPL,3571,0,4,NaN,0,314,APPLE INC,NaN,A,0
1,101594,20150630,AAPL 150702C105000,1,20150702,20150630.0,C,105000,20.55,20.65,16,7,1.352380,0.966076,0.006003,0.699296,-86.50655,108201234,1,0,100,0,125.42603,w,NaN,NaN,3783310,AAPL,3571,0,4,NaN,0,314,APPLE INC,NaN,A,0
2,101594,20150630,AAPL 150702C107000,1,20150702,20150630.0,C,107000,18.25,18.80,10,0,1.115425,0.975360,0.005581,0.536270,-54.77605,108249631,1,0,100,0,125.42603,w,NaN,NaN,3783310,AAPL,3571,0,4,NaN,0,314,APPLE INC,NaN,A,0
3,101594,20150630,AAPL 150702C108000,1,20150702,NaN,C,108000,17.05,17.80,0,0,NaN,NaN,NaN,NaN,NaN,108249632,1,0,100,0,125.42603,w,NaN,NaN,3783310,AAPL,3571,0,4,NaN,0,314,APPLE INC,NaN,A,0
4,101594,20150630,AAPL 150702C109000,1,20150702,20150630.0,C,109000,16.55,16.65,286,366,1.112591,0.959576,0.008420,0.807882,-82.13522,108051023,1,0,100,0,125.42603,w,NaN,NaN,3783310,AAPL,3571,0,4,NaN,0,314,APPLE INC,NaN,A,0


In [23]:
get_dividend_df("AAPL").head()

,exOrEffDate,type,amount,declarationDate,recordDate,paymentDate,Unnamed: 6
0,11/07/2019,Cash,$0.77,10/30/2019,11/11/2019,11/14/2019,NaN
1,08/09/2019,Cash,$0.77,07/30/2019,08/12/2019,08/15/2019,NaN
2,05/10/2019,Cash,$0.77,04/30/2019,05/13/2019,05/16/2019,NaN
3,02/08/2019,Cash,$0.73,01/29/2019,02/11/2019,02/14/2019,NaN
4,11/08/2018,Cash,$0.73,11/01/2018,11/12/2018,11/15/2018,NaN


## Step 1: Merge 3 Tables Together (Code)

In [138]:
def merge(ticker):
    dividends = get_dividend_df(ticker)
    opts = get_opt_df(ticker)
    daily = get_stonks_df(ticker)#get_daily_df(ticker)
    temp = pd.merge(dividends, daily, on='DATE', how='outer')
    last = pd.merge(opts, temp, on='DATE')
    return last

In [139]:
def merge_before(ticker, days):
    dividends = get_dividend_df(ticker)
    dividends['DATE'] = dividends.apply (lambda row: row['DATE'] - timedelta(days=days), axis=1)
    opts = get_opt_df(ticker)
    daily = get_stonks_df(ticker)#get_daily_df(ticker)
    
    
    temp = pd.merge(dividends, daily, on='DATE', how='outer')
    last = pd.merge(opts, temp, on='DATE')
    return last

In [182]:
TAX = .25
def calculate_arb(row):
    #Div*(1-tax) - Cost_option < Underlying - Strike
    Div = float(row['amount'][1:])*(1-TAX)
    Cost_Option = row['best_offer']
    Call = row['cp_flag'] == 'C'
    Underlying = row['PRC']
    Strike = row['strike_price']/1000
    ITM = Strike > Underlying
    
    #print(Div, Cost_Option, Call, Underlying, Strike)
    

    if not Call and ITM:
        return Strike + Div - Underlying - Cost_Option
    return 0

In [183]:
def percent_arb(row):
    #Div * (1 - tax) / (Underlying + Cost_Option)
    Div = float(row['amount'][1:])*(1-TAX)
    Cost_Option = row['best_offer']
    Call = row['cp_flag'] == 'C'
    Underlying = row['PRC']
    Strike = row['strike_price']/1000
    ITM = Strike > Underlying
    
    #print(Div, Cost_Option, Call, Underlying, Strike)
    

    if not Call and ITM:
        return Div/(Underlying + Cost_Option)
    return 0

In [184]:
def get_arb(ticker):
    try:
        df = merge_before(ticker,1)
        filtered = df[df.exOrEffDate.notnull()]
        filtered['ARB'] = filtered.apply(lambda row: calculate_arb(row), axis=1)
        filtered['PNL'] = filtered.apply(lambda row: percent_arb(row), axis=1)
        return filtered[filtered['ARB'] > 0]
    except:
        print("Incomplete Data for {}".format(ticker))
        return 0

In [ ]:
%%capture --no-stdout

arbs = get_arb("IBM")
for ticker in stonks.TICKER.unique()[:200]:
    print(ticker)
    #ticker_arb = get_arb(ticker)
    try:
        data = get_arb(ticker)
        """
        if not data.empty:
            print(data)
            break
        """
        arbs = pd.concat([arbs, data])
    except:
        pass
    #print(get_arb(ticker))

JJSF
DGSE
Incomplete Data for DGSE
DGC
Incomplete Data for DGC
PLXS
Incomplete Data for PLXS
HGR
Incomplete Data for HGR
nan
Incomplete Data for nan
HNGR
Incomplete Data for HNGR
SUNW
Incomplete Data for SUNW
JAVA
Incomplete Data for JAVA
ORCL
MSFT
Incomplete Data for MSFT
TW
TROW
HON
AMRC
Incomplete Data for AMRC
AKN
Incomplete Data for AKN
AKRX
Incomplete Data for AKRX
TECD
Incomplete Data for TECD
RGEN
Incomplete Data for RGEN
AT
LSBX
Incomplete Data for LSBX
BWC
Incomplete Data for BWC
BWXT
FO
Incomplete Data for FO
BEAM
Incomplete Data for BEAM
BWINB
Incomplete Data for BWINB
PTVCB
INDB
DMC
Incomplete Data for DMC
DSS
Incomplete Data for DSS
AVAN
Incomplete Data for AVAN
CLDX
Incomplete Data for CLDX
HVT
CY
TMP
BCPC
XOMA
Incomplete Data for XOMA
AMAG
Incomplete Data for AMAG
TCB
Incomplete Data for TCB
TCF
ASTE


In [179]:
temp = arbs[arbs["volume"] > 0]

In [181]:
temp

,secid,date_x,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,optionid,cfadj,am_settlement,contract_size,ss_flag,forward_price,expiry_indicator,root,suffix,cusip,ticker,sic,index_flag,exchange_d,class,issue_type,industry_group,issuer,div_convention,exercise_style,am_set_flag,DATE,exOrEffDate,type,amount,declarationDate,recordDate,paymentDate,Unnamed: 6,PERMNO,date_y,TICKER,BIDLO,ASKHI,PRC,BID,ASK,OPENPRC,RETX,ARB
85366,105785,20161115,HON1 161118P115000,1,20161118,20161115.0,P,115000,1.79,2.29,2,122,NaN,NaN,NaN,NaN,NaN,113175529,1,0,100,1,-99.990000,NaN,NaN,NaN,43851610,HON,5099,0,1,NaN,0,622.0,HONEYWELL INTL INC,NaN,A,0,2016-11-15,11/16/2016,Cash,$0.665,10/28/2016,11/18/2016,12/09/2016,NaN,10145.0,20161115,HON,112.09000,113.44,113.120,113.11000,113.12000,113.06000,0.000708,0.08875
229062,105785,20180221,HON 180223P157500,1,20180223,20180221.0,P,157500,2.13,4.00,12,46,NaN,NaN,NaN,NaN,NaN,118622701,1,0,100,0,153.317531,w,NaN,NaN,43851610,HON,5099,0,1,NaN,0,622.0,HONEYWELL INTL INC,NaN,A,0,2018-02-21,02/22/2018,Cash,$0.745,02/08/2018,02/23/2018,03/09/2018,NaN,10145.0,20180221,HON,154.03000,156.86,154.050,154.10001,154.11000,154.98000,-0.001879,0.00875
229063,105785,20180221,HON 180223P160000,1,20180223,20180221.0,P,160000,5.80,6.50,7,115,NaN,NaN,NaN,NaN,NaN,118622702,1,0,100,0,153.317531,w,NaN,NaN,43851610,HON,5099,0,1,NaN,0,622.0,HONEYWELL INTL INC,NaN,A,0,2018-02-21,02/22/2018,Cash,$0.745,02/08/2018,02/23/2018,03/09/2018,NaN,10145.0,20180221,HON,154.03000,156.86,154.050,154.10001,154.11000,154.98000,-0.001879,0.00875
229065,105785,20180221,HON 180223P165000,1,20180223,20180221.0,P,165000,9.15,11.45,1,31,NaN,NaN,NaN,NaN,NaN,118622704,1,0,100,0,153.317531,w,NaN,NaN,43851610,HON,5099,0,1,NaN,0,622.0,HONEYWELL INTL INC,NaN,A,0,2018-02-21,02/22/2018,Cash,$0.745,02/08/2018,02/23/2018,03/09/2018,NaN,10145.0,20180221,HON,154.03000,156.86,154.050,154.10001,154.11000,154.98000,-0.001879,0.05875
229142,105785,20180221,HON 180302P162500,1,20180302,20180221.0,P,162500,8.35,9.00,10,62,NaN,NaN,NaN,NaN,NaN,118689071,1,0,100,0,153.361875,w,NaN,NaN,43851610,HON,5099,0,1,NaN,0,622.0,HONEYWELL INTL INC,NaN,A,0,2018-02-21,02/22/2018,Cash,$0.745,02/08/2018,02/23/2018,03/09/2018,NaN,10145.0,20180221,HON,154.03000,156.86,154.050,154.10001,154.11000,154.98000,-0.001879,0.00875
366094,105785,20181114,HON1 181116P155000,1,20181116,20181114.0,P,155000,3.00,4.00,10,232,NaN,NaN,NaN,NaN,NaN,124001306,1,0,100,1,-99.990000,NaN,NaN,NaN,43851610,HON,5099,0,1,NaN,0,622.0,HONEYWELL INTL INC,NaN,A,0,2018-11-14,11/15/2018,Cash,$0.82,09/28/2018,11/16/2018,12/07/2018,NaN,10145.0,20181114,HON,147.21001,149.19,147.530,147.53999,147.57001,148.64999,-0.001016,4.08500
366095,105785,20181114,HON1 181116P160000,1,20181116,20181114.0,P,160000,7.30,9.50,10,395,NaN,NaN,NaN,NaN,NaN,124001307,1,0,100,1,-99.990000,NaN,NaN,NaN,43851610,HON,5099,0,1,NaN,0,622.0,HONEYWELL INTL INC,NaN,A,0,2018-11-14,11/15/2018,Cash,$0.82,09/28/2018,11/16/2018,12/07/2018,NaN,10145.0,20181114,HON,147.21001,149.19,147.530,147.53999,147.57001,148.64999,-0.001016,3.58500
366096,105785,20181114,HON1 181116P165000,1,20181116,20181114.0,P,165000,12.30,14.35,3,1152,NaN,NaN,NaN,NaN,NaN,124001308,1,0,100,1,-99.990000,NaN,NaN,NaN,43851610,HON,5099,0,1,NaN,0,622.0,HONEYWELL INTL INC,NaN,A,0,2018-11-14,11/15/2018,Cash,$0.82,09/28/2018,11/16/2018,12/07/2018,NaN,10145.0,20181114,HON,147.21001,149.19,147.530,147.53999,147.57001,148.64999,-0.001016,3.73500
366141,105785,20181114,HON1 181221P155000,1,20181221,20181114.0,P,155000,5.05,6.40,11,278,NaN,NaN,NaN,NaN,NaN,120217168,1,0,100,1,-99.990000,NaN,NaN,NaN,43851610,HON,5099,0,1,NaN,0,622.0,HONEYWELL INTL INC,NaN,A,0,2018-11-14,11/15/2018,Cash,$0.82,09/28/2018,11/16/2018,12/07/2018,NaN,10145.0,20181114,HON,147.21001,149.19,147.530,147.53999,147.57001,148.64999,-0.001016,1.68500
366142,105785,20181114,HON1 181221P160000,1,20181221,20181114.0,P,160000,8.50,9.80,3,371,NaN,NaN,NaN,NaN

In [180]:
calculate_arb(temp.iloc[-1])

0.21000000000000002 1.35 False 26.46 55.0


27.4